In [1]:
import numpy as np
from os import listdir
import pandas as pd
import cv2

In [2]:
images = [x for x in listdir("editSpectograms") if ".jpg" in x]

In [3]:
df = pd.DataFrame(columns=["File Name", "Number of Events", "Duration of Events", "Frequency Change of Events"])

In [4]:
df.head()

,File Name,Number of Events,Duration of Events,Frequency Change of Events


In [5]:
#Converts the pixel values in our slice to frequencies based on the scale
def intToFrequency(number, bottom_frequency, top_frequency, max_value):
    difference = top_frequency - bottom_frequency
    percentage = number / max_value
    return round(top_frequency - percentage * difference - 2, 2)

In [8]:
for image in images:
    filename = "editSpectograms/" + image
    im = cv2.imread(filename)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.blur(im,(6,3))
    im = cv2.Canny(im, 90, 300)
    mask = cv2.inRange(im, 100, 255)
    kernel = np.ones((5, 5), np.uint8)
    # Erode / dilate to make label appear "boxy"
    boxy = cv2.erode(mask, kernel, iterations=0)
    boxy = cv2.dilate(boxy, kernel, iterations=1)
    ##Create the boxes
    contours, _ = cv2.findContours(boxy, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    width_list = []
    frequency_list = []
    for contour in contours:
        cv2.drawContours(im, contour, -1, (0, 255, 0), 3)
        boundRect = cv2.boundingRect(contour)
        color = (180, 180, 180)
        cv2.rectangle(boxy, (int(boundRect[0]), int(boundRect[1])), 
        (int(boundRect[0]+boundRect[2]), int(boundRect[1]+boundRect[3])), color, 2)
        cv2.rectangle(im, (int(boundRect[0]), int(boundRect[1])), 
        (int(boundRect[0]+boundRect[2]), int(boundRect[1]+boundRect[3])), color, 2)
        
        width = boundRect[2]
        height= boundRect[3]
        highFreq = intToFrequency(boundRect[1], 16.2, 26.62, 90)
        lowFreq = intToFrequency((boundRect[1] + height), 16.2, 26.62, 90)
        frequency_list.append(str(lowFreq) + "-" + str(highFreq) + " Hz")
        width_list.append(width)
    elements = len(width_list)
    s2 = pd.Series([image, elements, width_list, frequency_list], index=['File Name', 'Number of Events', 'Duration of Events', 'Frequency Change of Events'])
    df = df.append(s2, ignore_index=True)

In [12]:
df.head(100)

,File Name,Number of Events,Duration of Events,Frequency Change of Events
0,e_1_18_57_30.jpg,6,"[32, 38, 26, 5, 35, 40]","[15.82-17.33 Hz, 15.82-17.56 Hz, 17.09-18.37 H..."
1,e_6_17_17_30.jpg,0,[],[]
2,e_2_21_35_00.jpg,0,[],[]
3,e_4_01_35_00.jpg,1,[126],[18.83-20.8 Hz]
4,e_3_00_40_00.jpg,0,[],[]
5,e_1_02_57_30.jpg,0,[],[]
6,e_2_22_37_30.jpg,0,[],[]
7,e_1_01_52_30.jpg,0,[],[]
8,e_4_23_00_00.jpg,0,[],[]
9,e_2_18_30_00.jpg,0,[],[]
